In [35]:
from match_functions import get_match_database
data = get_match_database('2025-03-01', '2025-03-15', leagues = ["Ligue 1"], save = True, add = False)

Extraction des liens...



100%|██████████| 15/15 [02:05<00:00,  8.38s/it]


Chargement de la base de données...
 
Extraction terminée en  4.52 minutes


In [38]:
import ace_tools_open as tools
tools.display_dataframe_to_user("Data", data)

Data


# Links Whoscored

In [ ]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm
import time


def get_link(soup):
    code = soup.find_all("a")
    for c in code:
        if "Centre du Match" in c.text:
            if c.get("href") != "#":
                link = c.get("href")
            else:
                link = "None"
            break

    return link


def sep_list(url):
    if url != "None":
        pattern = r"/matches/(\d+)/live/([a-z-]+)-([a-z0-9-]+)-(\d{4}-\d{4})-(.+)"
        match = re.match(pattern, url)
        
        if match:
            numero = match.group(1)
            pays = match.group(2).replace("-", " ").title()
            ligue = match.group(3).replace("-", " ").title()
            annee = match.group(4)
            clubs = match.group(5).replace("-", " ").title()
            return numero, url#,pays, ligue, annee, clubs
        return None
    
    

def get_link_whoscored(range_down, range_up, path):

    number_list = list(range(range_down, range_up))
    pre_links = []
    for el in number_list:
        pre_links.append("https://fr.whoscored.com/matches/" + f"{el}")

    all_parsed_matches = pd.DataFrame()
    all_not_available_parsed_matches = pd.DataFrame()
    all_errors_parsed_matches = pd.DataFrame()

    links = []
    errors = []
    not_available = []

    i = 0
    j = 0
    k = 0

    options = Options()
    options.add_argument("--headless")  # Mode headless activé
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)

    try:
        for pre_link in tqdm(pre_links):
            try:
                driver.get(pre_link)
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                link = get_link(soup)
                if link == "None":
                    not_available.append(pre_link)
                    j+=1
                else:
                    links.append(link)
                    i+=1

            except Exception as e:
                print(f"❌ Erreur lors de la récupération de la page : {e}")
                errors.append(pre_link)
                k += 1

            if k % 10 == 0:
                #error
                errors_parsed_matches = pd.DataFrame(errors)#, columns=["link"])
                all_errors_parsed_matches = pd.concat([all_errors_parsed_matches, errors_parsed_matches])
                all_errors_parsed_matches.to_csv("all_errors_whoscored.csv", index=False)
                j=0
                errors = []

            if i % 10 == 0:
                # Link
                parsed_matches = [sep_list(url) for url in links]
                parsed_matches = pd.DataFrame(parsed_matches)#, columns=["numéro", "link"])
                all_parsed_matches = pd.concat([all_parsed_matches, parsed_matches])
                all_parsed_matches.to_csv("all_links_whoscored.csv", index=False)
                i = 0
                links = []

            if j % 10 == 0:
                # not_available
                not_available_parsed_matches = pd.DataFrame(not_available)#, columns=["link"])
                all_not_available_parsed_matches = pd.concat([all_not_available_parsed_matches, not_available_parsed_matches])
                all_not_available_parsed_matches.to_csv("all_not_available_whoscored.csv", index=False)
                j=0
                not_available = []
            
        parsed_matches = [sep_list(url) for url in links]
        parsed_matches = pd.DataFrame(parsed_matches)
        not_available_parsed_matches = pd.DataFrame(not_available)
        errors_parsed_matches = pd.DataFrame(errors)

        all_parsed_matches = pd.concat([all_parsed_matches, parsed_matches])
        all_errors_parsed_matches = pd.concat([all_errors_parsed_matches, errors_parsed_matches])
        all_not_available_parsed_matches = pd.concat([all_not_available_parsed_matches, not_available_parsed_matches])

        if path != "None":
            all_parsed_matches = pd.read_csv(f"{path}all_links_whoscored.csv")
            all_not_available_parsed_matches = pd.read_csv(f"{path}all_errors_whoscored.csv")
            all_errors_parsed_matches = pd.read_csv(f"{path}all_not_available_whoscored.csv")

            

        all_parsed_matches.to_csv("all_links_whoscored.csv", index=False)
        all_errors_parsed_matches.to_csv("all_errors_whoscored.csv", index=False)
        all_not_available_parsed_matches.to_csv("all_not_available_whoscored.csv", index=False)

    finally:
        driver.quit()



In [2]:
get_link_whoscored(1820000, 1820050, "None", "None", "None")

100%|██████████| 50/50 [01:03<00:00,  1.27s/it]


In [4]:
get_link_whoscored(1820050, 1820100, "all_links_whoscored.csv", "all_errors_whoscored.csv", "all_not_available_whoscored.csv")

100%|██████████| 50/50 [01:13<00:00,  1.47s/it]


In [5]:
links = pd.read_csv("all_links_whoscored.csv")
import ace_tools_open as tools
tools.display_dataframe_to_user("Data", links)

Data


In [28]:
error = pd.read_csv("all_errors_whoscored.csv")
error

EmptyDataError: No columns to parse from file

In [29]:
not_available = pd.read_csv("all_not_available_whoscored.csv")
not_available

,0
0,https://fr.whoscored.com/matches/1820004
1,https://fr.whoscored.com/matches/1820006
2,https://fr.whoscored.com/matches/1820008
3,https://fr.whoscored.com/matches/1820010
4,https://fr.whoscored.com/matches/1820011
5,https://fr.whoscored.com/matches/1820012
6,https://fr.whoscored.com/matches/1820014
7,https://fr.whoscored.com/matches/1820016
8,https://fr.whoscored.com/matches/1820018
9,https://fr.whoscored.com/matches/1820020


# Current month link Whoscored

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import time

def get_link_current_month(url):

    options = Options()
    options.add_argument("--headless")  # Mode headless activé
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)

    all_links = []

    try:
        driver.get(url)
        
        # Accepter les cookies si la bannière apparaît
        try:
            accept_cookies = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "qc-cmp-cleanslate"))
            )
            driver.execute_script("arguments[0].click();", accept_cookies)
            time.sleep(2)
        except Exception:
            print("Bannière de cookies non détectée.")
        
        time.sleep(2)  # Attendre le chargement de la page
        
        # Récupération du HTML
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        # Extraction des liens
        code = soup.find_all("div", class_="Match-module_right_oddsOn__o-ux-")
        for c in code:
            if c.find("a") is not None:
                all_links.append("https://fr.whoscored.com" + c.find("a").get("href"))
            
    finally:
        driver.quit()

    return all_links


In [2]:
url = "https://fr.whoscored.com/regions/74/tournaments/22/seasons/10329/stages/23414/fixtures/france-ligue-1-2024-2025"
links = get_link_current_month(url)
links

['https://fr.whoscored.com/matches/1824845/live/france-ligue-1-2024-2025-saint-etienne-nice',
 'https://fr.whoscored.com/matches/1824846/live/france-ligue-1-2024-2025-lens-le-havre',
 'https://fr.whoscored.com/matches/1824841/live/france-ligue-1-2024-2025-paris-saint-germain-lille',
 'https://fr.whoscored.com/matches/1824843/live/france-ligue-1-2024-2025-lyon-brest',
 'https://fr.whoscored.com/matches/1824844/live/france-ligue-1-2024-2025-montpellier-rennes',
 'https://fr.whoscored.com/matches/1824849/live/france-ligue-1-2024-2025-angers-toulouse',
 'https://fr.whoscored.com/matches/1824847/live/france-ligue-1-2024-2025-auxerre-strasbourg',
 'https://fr.whoscored.com/matches/1824842/live/france-ligue-1-2024-2025-marseille-nantes',
 'https://fr.whoscored.com/matches/1824853/live/france-ligue-1-2024-2025-toulouse-monaco',
 'https://fr.whoscored.com/matches/1824857/live/france-ligue-1-2024-2025-rennes-paris-saint-germain',
 'https://fr.whoscored.com/matches/1824856/live/france-ligue-1-202

# Rate from Whoscored

In [ ]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime


def get_event(soup):
    code = soup.find_all("a", class_="team-link")
    team_dom = code[0].text
    team_ext = code[1].text

    c = soup.find_all("div", class_="info-block cleared")[2].text.replace("Coup d'envoi:", "").replace("Date:Sun, ", "/").split("/")
    date_str = c[1]
    date_obj = datetime.strptime(date_str, '%d-%b-%y')  # Convertir en objet datetime
    formatted_date = date_obj.strftime('%d/%m/%Y')  # Reformater en "01/03/2025"
    date = formatted_date
    heure = c[0]
    stade = soup.find_all("span", class_="value")[0].text
    arbitre = soup.find_all("span", class_="value")[2].text.split(". ")[1]
    affluence = int(soup.find_all("span", class_="value")[1].text.replace(",", ""))

    return date, heure, team_dom, team_ext, affluence, stade



def get_page_rate(url):
    options = Options()
    options.add_argument("--headless")  # Mode headless activé
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)
    
    try:
        driver.get(url)

        # Attendre que les stats des joueurs apparaissent ET contiennent du texte
        WebDriverWait(driver, 15).until(
            lambda d: any(e.text.strip() for e in d.find_elements(By.CLASS_NAME, "player-stat-value"))
        )

        html = driver.page_source
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    return soup


def get_rate(url):
    soup = get_page_rate(url)

    # Extraction des notes
    rate = [c.text.strip() if c.text.strip() else "NaN" for c in soup.find_all("span", class_="player-stat-value")]

    # Extraction des noms des joueurs
    player = [c.text.strip() for c in soup.find_all("span", class_="player-name")][:len(rate)]
    n = len(player)

    # Event
    match = re.search(r'/live/[^/-]+-([^/]+)-(\d{4}-\d{4})', url)
    championnat = [match.group(1).replace('-', ' ').title()] * n
    saison = [match.group(2)] * n

    date, heure, team_dom, team_ext, affluence, stade = get_event(soup)

    date = [date] * n
    heure = [heure] * n
    team_dom = [team_dom] * n
    team_ext = [team_ext] * n
    affluence = [affluence] * n
    stade = [stade] * n

    # Création du DataFrame
    df_rate = pd.DataFrame({'championnat':championnat, 'saison': saison, 'date': date, 'heure': heure, 'stade':stade, 'Equipe1': team_dom, 'Equipe2': team_ext, 'affluence':affluence, 'joueur': player, 'rate': rate})

    return df_rate


In [38]:
# 🔥 Test
url = "https://fr.whoscored.com/matches/1824847/live/france-ligue-1-2024-2025-auxerre-strasbourg"
df = get_rate(url)
import ace_tools_open as tools
tools.display_dataframe_to_user("", df)